In [179]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Loading dataset:

In [180]:
import pandas as pd
df_full = pd.read_csv("../input/liver-train-compe/train_dataset.csv")
df_full

Separating a small slice of data as validation set.\
This is meant for tackling **data leakage** due to train-test contamination.

In [181]:
df = df_full.iloc[:6500,:].copy()
df_val = df_full.iloc[6500:,:].copy()    
df_val.reset_index(inplace = True)
print("Shape of new dataframes - {} , {}".format(df.shape, df_val.shape))

Checking number of missing values per column:

In [182]:
col_list = df.columns
ukw_cols = []
for i in col_list:
    if df[i].value_counts().sum() != len(df):
        ukw_val = len(df) - df[i].value_counts().sum()
        ukw_cols.append((i,ukw_val))
df2 = pd.DataFrame(ukw_cols, columns=['Feature','No. of missing values'])
df2

In [183]:
s = df.isnull().sum().sum()
missing_percent = (s*100)/(len(df)*(len(df.columns)-4))     #Dropping ID, N_Days, Status and Stage from total data consideration
print("Percentage of missing data in the whole dataset: {}%.".format(missing_percent.round(2)))

Since there are so many missing values, filtering the dataset by dropping samples/rows with more than 2 missing values.

In [184]:
df.dropna(axis= 0, inplace= True, thresh = 18)
df

# ENCODING OPERATION 
Encoding categorical columns using Ordinal Encoding in the following way:
* Spiders:       Y = 1 and N = 0
* Hepatomegaly:  Y = 1 and N = 0
* Ascites:       Y = 1 and N = 0
* Drug:          D-penicillamine = 0 and Placebo = 1  (reason for this particular encoding is explained later)
* Edema:         N = -1, S = 0, Y = 1
* Status:        D = -1, CL = 0, C = 1

For Sex, One Hot Encoding is performed.

In [185]:
def EncodeAll(df):
    from sklearn.preprocessing import OrdinalEncoder

    df["NaN_Drug"] = df["Drug"].isna()
    df_copy = df.copy()
    ordinal_encoder = OrdinalEncoder()
    col_dict = {"Y": 1, "N": 0}
    df_copy["Spiders"].replace(col_dict, inplace=True)
    df_copy["Hepatomegaly"].replace(col_dict, inplace=True)
    df_copy["Ascites"].replace(col_dict, inplace=True)
    df_copy["Drug"].replace({"D-penicillamine": 0, "Placebo": 1}, inplace = True)
    df_copy["Edema"].replace({"N": -1, "S": 0, "Y": 1}, inplace = True)
    df_copy["Status"].replace({"D": -1, "CL": 0, "C": 1}, inplace = True)


    from sklearn.preprocessing import OneHotEncoder


    OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
    df_OHE = pd.DataFrame(OH_encoder.fit_transform(df_copy[["Sex"]]))
    df_OHE.index = df_copy.index

    df_encoded = df_copy.drop(["Sex"], axis=1)
    encoded_cols = list(OH_encoder.get_feature_names(["Sex"]))
    df_encoded[encoded_cols] = OH_encoder.transform(df_copy[["Sex"]])

    return df_encoded

df_encoded = EncodeAll(df)
df_encoded_val = EncodeAll(df_val)


Due to the presence of ambiguity (missing values) of which drug was administered to patients, a separate column has been created which keeps track of this ambiguity in form of boolean data.\
This column represents which all samples/patients previously had `NaN` in Drug column.

# IMPUTATION OPERATION
Number of missing values per feature:


In [186]:
df.isna().sum()

In [187]:
df_val.isna().sum()

Using KNNImputer for imputing the missing values in the features. \
It is a more useful method which works on the basic approach of the KNN algorithm rather than the naive approach of filling all the values with mean or the median.

In [188]:
#Imputation
import numpy as np
from sklearn.impute import KNNImputer
import pandas as pd

def ImputeAll(df_encoded):
    imputer = KNNImputer(n_neighbors =5)                              #Mess around with n_neighbors to improve accuracy in the end model
    imputed_data = imputer.fit_transform(df_encoded) 
    df_imputed = pd.DataFrame(imputed_data)
    df_imputed.columns = df_encoded.columns
    del imputer

    return df_imputed

df_imputed = ImputeAll(df_encoded)
df_imputed_val = ImputeAll(df_encoded_val)

In [189]:
df_imputed.isna().sum()

In [190]:
df_imputed_val.isna().sum()

All missing values have been accounted for.

In [191]:
df_imputed.describe()

In [192]:
df_imputed_val.describe()

# FEATURE EXPLORATION
Exploring and analysing features using visualization and statistics against target to draw insights about correlations.

In [193]:
import matplotlib.pyplot as plt
import seaborn as sns

a = df_imputed["Stage"].value_counts()
from matplotlib import pyplot as plt
a = dict(a)
stages = list(a.keys())
counts = list(a.values())
colors = ['r', 'y', 'royalblue', 'g']
fig = plt.figure(figsize = (10, 5))

fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
ax.axis('equal')
ax.pie(counts, labels = stages,autopct='%1.2f%%', colors= colors, wedgeprops = {"edgecolor" : "black",'linewidth': 1,'antialiased': True})
plt.legend()
plt.title("Target Proportions")
plt.show()

**Target Class** \
People with early-stage cirrhosis of the liver usually don't have symptoms. \
Natually, those people would not get themselves checked up and by the time they show symptoms and get tested, their liver cirrhosis would have progressed to a higher stage and therefore the target class seems to be imbalanced. 

In [194]:
df_1 = df_imputed[df_imputed['Stage']==1]
df_2 = df_imputed[df_imputed['Stage']==2]
df_3 = df_imputed[df_imputed['Stage']==3]
df_4 = df_imputed[df_imputed['Stage']==4]

plt.figure(figsize=(20,10))
f, axes = plt.subplots(nrows = 2, ncols = 2, sharex=True, sharey = True, figsize = (10, 10))
axes[0][0].scatter(df_1["Bilirubin"],df_1["Alk_Phos"], s = 5, color = 'g')
axes[0][0].set_xlabel('Bilirubin', labelpad = 5)
axes[0][0].set_ylabel('Alkaline Phosphatase', labelpad = 5)
axes[0][0].set_title('Stage 1')


axes[0][1].scatter(df_2["Bilirubin"],df_2["Alk_Phos"], s= 5, color = 'royalblue')
axes[0][1].set_xlabel('Bilirubin', labelpad = 5)
axes[0][1].set_ylabel('Alkaline Phosphatase', labelpad = 5)
axes[0][1].set_title('Stage 2')

axes[1][0].scatter(df_3["Bilirubin"],df_3["Alk_Phos"], s= 5, color = 'goldenrod')
axes[1][0].set_xlabel('Bilirubin', labelpad = 5)
axes[1][0].set_ylabel('Alkaline Phosphatase', labelpad = 5)
axes[1][0].set_title('Stage 3')

axes[1][1].scatter(df_4["Bilirubin"],df_4["Alk_Phos"], s= 5, color = 'r')
axes[1][1].set_xlabel('Bilirubin', labelpad = 5)
axes[1][1].set_ylabel('Alkaline Phosphatase', labelpad = 5)
axes[1][1].set_title('Stage 4')

f.suptitle("Bilirubin x Alkaline Phosphatase", fontsize=15)

In [195]:
plt.figure(figsize=(7,7))
plt.scatter(df_val["Bilirubin"],df_val["Alk_Phos"], s= 5, color = 'deeppink')
plt.xlabel("Bilirubin")
plt.ylabel("Alkaline Phosphatase")
plt.title("Bilirubin x Alkaline Phosphatase for (Validation)")
plt.show()

**Bilirubin**\
Bilirubin is the yellow pigment extracted from old blood cells by the liver and excreeted with the stool. Studies have shown that increase in Bilirubin in blood accounts for liver malfunction. \
The given data shows that Bilirubin value in blood is directly proportional to the cirrhosis of the liver.

**Alkaline Phosphatase**\
Alkaline Phosphatase (ALP) is produced when there is bone damage or liver damage. To find the right source, a follow-up Bilirubin test is conducted. \
If the Bilirubin value is within the normal range then the cause for ALP production would be some bone damage and if the value of Bilirubin is high, then it is concluded that the ALP is in fact produced by damaged liver.

In [196]:
plt.figure(figsize=(20,10))
f, axes = plt.subplots(nrows = 2, ncols = 2, sharex=True, sharey = True, figsize = (10, 10))
axes[0][0].scatter(df_1["Tryglicerides"],df_1["Alk_Phos"], s = 5, color = 'g')
axes[0][0].set_xlabel('Tryglicerides', labelpad = 5)
axes[0][0].set_ylabel('Alkaline Phosphatase', labelpad = 5)
axes[0][0].set_title('Stage 1')


axes[0][1].scatter(df_2["Tryglicerides"],df_2["Alk_Phos"], s= 5, color = 'royalblue')
axes[0][1].set_xlabel('Tryglicerides', labelpad = 5)
axes[0][1].set_ylabel('Alkaline Phosphatase', labelpad = 5)
axes[0][1].set_title('Stage 2')

axes[1][0].scatter(df_3["Tryglicerides"],df_3["Alk_Phos"], s= 5, color = 'y')
axes[1][0].set_xlabel('Tryglicerides', labelpad = 5)
axes[1][0].set_ylabel('Alkaline Phosphatase', labelpad = 5)
axes[1][0].set_title('Stage 3')

axes[1][1].scatter(df_4["Tryglicerides"],df_4["Alk_Phos"], s= 5, color = 'r')
axes[1][1].set_xlabel('Tryglicerides', labelpad = 5)
axes[1][1].set_ylabel('Alkaline Phosphatase', labelpad = 5)
axes[1][1].set_title('Stage 4')

f.suptitle("Tryglicerides x Alkaline Phosphatase", fontsize=15)

In [197]:
plt.figure(figsize=(7,7))
plt.scatter(df_val["Bilirubin"],df_val["Alk_Phos"], s= 5, color = 'deeppink')
plt.xlabel("Tryglicerides")
plt.ylabel("Alkaline Phosphatase")
plt.title("Tryglicerides x Alkaline Phosphatase for (Validation)")
plt.show()

**Triglycerides**\
Triglycerides are a type of fat (lipid) found in the blood and the liver is the central organ for fatty acid metabolism. If there is presence of high levels of Triglycerides, it indicates that the liver is not functioning properly. With the progression of liver cirrhosis, higher levels of Triglycerides are found in the blood.\
Triglycerides test can also be used to determine the source of ALP production in the body and the graphs dictate that it is produced due to cirrhosis of liver.


In [198]:
plt.figure(figsize=(20,10))
f, axes = plt.subplots(nrows = 2, ncols = 2, sharex=True, sharey = True, figsize = (10, 10))
axes[0][0].scatter(df_1["Bilirubin"],df_1["SGOT"], s = 5, color = 'g')
axes[0][0].set_xlabel('Bilirubin', labelpad = 5)
axes[0][0].set_ylabel('SGOT', labelpad = 5)
axes[0][0].set_title('Stage 1')


axes[0][1].scatter(df_2["Bilirubin"],df_2["SGOT"], s = 5, color = 'royalblue')
axes[0][1].set_xlabel('Bilirubin', labelpad = 5)
axes[0][1].set_ylabel('SGOT', labelpad = 5)
axes[0][1].set_title('Stage 2')

axes[1][0].scatter(df_3["Bilirubin"],df_3["SGOT"], s = 5, color = 'y')
axes[1][0].set_xlabel('Bilirubin', labelpad = 5)
axes[1][0].set_ylabel('SGOT', labelpad = 5)
axes[1][0].set_title('Stage 3')

axes[1][1].scatter(df_4["Bilirubin"],df_4["SGOT"], s = 5, color = 'r')
axes[1][1].set_xlabel('Bilirubin', labelpad = 5)
axes[1][1].set_ylabel('SGOT', labelpad = 5)
axes[1][1].set_title('Stage 4')

f.suptitle("Bilirubin x SGOT", fontsize=15)

In [199]:
plt.figure(figsize=(7,7))
plt.scatter(df_val["Bilirubin"],df_val["Alk_Phos"], s= 5, color = 'deeppink')
plt.xlabel("Bilirubin")
plt.ylabel("SGOT")
plt.title("Bilirubin x SGOT for (Validation)")
plt.show()

**SGOT**  \
Serum glutamic oxaloacetic transaminase (SGOT), an enzyme that is normally present in liver and heart cells. SGOT is released into blood when the liver or heart is damaged. \
To find the right source, a follow-up Bilirubin test is conducted. If the Bilirubin value is within the normal range then the cause for SGOT production would be heart damage and if the value of Bilirubin is high, then it is concluded that the presence of high levels of SGOT is because of liver damage.\
The graphs drawn (above) support this fact.

**Prothrombin**\
Prothrombin is a protein made by the liver. It is one of several substances known as clotting (coagulation) factors. Prothrombin  time (in seconds) is the time taken for blood clotting. If the blood clots too slowy, it means the protein is being produced insufficiently by the liver, meaning the liver is malfunctioning/damaged. Average Prothrombin time in humans is 11 to 13.5 seconds. Generally, if the Prothrombin time is more than this range, it indicates severe lever damage/cirrhosis.

In [200]:
df_imputed.groupby("Stage")["Prothrombin"].mean()

In [201]:
df_imputed_val["Prothrombin"].mean()

The Prothrombin time data is inaccurate! Because the Prothrombin time given in the data is same for every stage of liver cirrhosis which is not possible. Realistically, Prothrombin time is supposed to increase (beyond 13.5 seconds) with the progression of liver cirrhosis. But the values given in the data tend to remain in the normal range.

**Urinary Copper**\
Urine is tested for the presence of copper. The copper urine test is used to determine the presence of Wilson disease, a sometimes fatal condition in which the buildup of excess copper damages the liver, and eventually the kidneys, eyes, and brain. Urinary copper excretion was found to be increased in patients with cholestasis, hepatitis and cirrhosis, but the penicillamine-induced increment was normal.

In [202]:
df_filter = df_imputed[df_imputed["Drug"] != 1]
df_filter.groupby("Stage")["Copper"].mean()

In [228]:
df_filter_val = df_imputed_val[df_imputed_val["Drug"] != 1]
df_filter_val["Copper"].mean()

As per the data, patients induced with penicillamine have stable urinary copper values and patients induced with placebo have more or less stagnant values.

**Drug**\
Drug column was labelled as D - Penicillamine = 0 and Placebo = 1 because:\
High concerntrations of Urinary Copper is a good indicator of the severity of liver damage. D - Penicillamine tends to control this increment in Copper concerntration and therefore masking the actual condition of liver. Hence, using Ordinal Encoding, D - Penicillamine has been weighted as 0 (zero importance).\
Whereas, Placebo has no actual treatment properties; Placebo does not control Copper levels in urine. This untreated Urinary Copper concerntration exposes the severity of liver damage. Therefore the Placebo has been weighted as 1 using Ordinal Encoding.

**Cholesterol**\
High-density lipoprotein (HDL) cholesterol and its major apolipoproteins have been shown to be reduced in cirrhosis. The given data gently reflects this trend.

In [203]:
df_imputed.groupby("Stage")["Cholesterol"].mean()

In [204]:
df_imputed_val["Cholesterol"].mean()

**Albumin**\
When the liver is damaged, Albumin is supposed to  decrease. Advanced cirrhosis is characterised by reduced albumin concentration as well as impaired albumin function as a result of specific structural changes and oxidative damage. The given data is shown to be low among all patients regardless of their corrhosis progression. Hence, Albumin data is redundant.

In [205]:
df.groupby(["Stage", "Edema"])["Albumin"].mean()

In [206]:
df_val.groupby(["Edema"])["Albumin"].mean()

**Edema**\
Edema is inflammation of muscles. It happens when blood vessels leak fluid into the tissues. Edema has many causes but one gets Cirrhosis related Edema when a follow-up Albumin test show low concerntrations. But the given data shows that patients have Cirrhosis regardless of whether they have Edema.

**Platelets**\
When liver is damaged, platelet production is said to be decreased and the given data does not reflect this trend.

In [207]:
df_imputed.groupby("Stage")["Platelets"].mean()

In [208]:
df_imputed_val["Platelets"].mean()

**Age**\
Visualising what age groups tend to get this disease.

In [209]:
df_1["Age"] = (df_1["Age"]/365).astype(int)
df_2["Age"] = (df_2["Age"]/365).astype(int)
df_3["Age"] = (df_3["Age"]/365).astype(int)
df_4["Age"] = (df_4["Age"]/365).astype(int)

plt.figure(figsize=(20,10))
f, axes = plt.subplots(nrows = 2, ncols = 2, sharex=True, sharey = True, figsize = (15, 15))
axes[0][0].hist(df_1["Age"], 20, facecolor='green', alpha=0.5, edgecolor='black')
axes[0][0].set_xticks(range(0,80,4))
axes[0][0].set_xlabel('Age Group', labelpad = 5)
axes[0][0].set_ylabel('Counts', labelpad = 5)
axes[0][0].set_title('Stage 1')

axes[0][1].hist(df_2["Age"], 20, facecolor='blue', alpha=0.5, edgecolor='black')
axes[0][1].set_xticks(range(0,80,4))
axes[0][1].set_xlabel('Age Group', labelpad = 5)
axes[0][1].set_ylabel('Counts', labelpad = 5)
axes[0][1].set_title('Stage 2')

axes[1][0].hist(df_3["Age"], 20, facecolor='yellow', alpha=0.5, edgecolor='black')
axes[1][0].set_xticks(range(0,80,4))
axes[1][0].set_xlabel('Age Group', labelpad = 5)
axes[1][0].set_ylabel('Counts', labelpad = 5)
axes[1][0].set_title('Stage 3')

axes[1][1].hist(df_4["Age"], 20, facecolor='red', alpha=0.5, edgecolor='black')
axes[1][1].set_xticks(range(0,80,4))
axes[1][1].set_xlabel('Age Group', labelpad = 5)
axes[1][1].set_ylabel('Counts', labelpad = 5)
axes[1][1].set_title('Stage 4')

f.suptitle("Age Distributions", fontsize=15)

In [210]:
df_val["Age"] = (df_val["Age"]/365).astype(int)
plt.figure(figsize=(7,7))
plt.hist(df_val["Age"], 20, facecolor='deeppink', alpha=0.5, edgecolor='black')
plt.show()

**Spider**\
Spider is an abnormal collection of blood vessels near the surface of the skin. Spiders were seen more commonly in patients with alcoholic cirrhosis than in those with non-alcoholic cirrhosis. So one might get cirrhosis regardless of whether they have Spiders. \
Therefore this data is not going to help predict target (y).

In [211]:
df.groupby(["Stage", "Spiders"])["ID"].count()

In [212]:
df_val.groupby(["Spiders"])["ID"].count()

**Hepatomegaly**\
Hepatomegaly is a condition where one has an enlarged liver. Hepatomegaly is considered as one of the potential complications of Liver Cirrhosis but not a definite indicator.

In [213]:
df.groupby(["Stage", "Hepatomegaly"])["ID"].count()

In [214]:
df_val.groupby(["Hepatomegaly"])["ID"].count()

**Ascites**\
Ascites is abdominal swelling caused due to accumulation of fluid in the abdomen. Mostly it is related to liver cirrhosis/liver cancer/kidney damage. Cirrhosis of the liver is the most common cause of ascites. But the given data points to the contrary. \
Therefore the data is not suitable for predicting target (y).

In [215]:
df.groupby(["Stage", "Ascites"])["ID"].count()

In [216]:
df_val.groupby(["Ascites"])["ID"].count()

# SCALING OPERATION
Using RobustScaler for removing the outliers and then using MinMaxScaler for preprocessing the dataset.

In [217]:
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import MinMaxScaler


def ScaleAll(df_imputed):
    cols = ['N_Days', 'Age', 'Ascites', 'Hepatomegaly', 'Spiders', 'Cholesterol', 'Albumin', 'Copper', 'Alk_Phos', 'SGOT', 'Tryglicerides', 'Platelets']
    temp = df_imputed[cols]
    df_scaled = df_imputed.copy()


    #Scaling selected columns
    scale = RobustScaler(with_centering=False, with_scaling=True)
    df_temp1 = pd.DataFrame(scale.fit_transform(temp))
    del scale
    scale = MinMaxScaler()
    df_temp2 = pd.DataFrame(scale.fit_transform(df_temp1))
    del scale

    df_temp2.columns = temp.columns
    df_scaled[cols] = df_temp2[cols]

    return df_scaled

df_scaled = ScaleAll(df_imputed)
df_scaled_val = ScaleAll(df_imputed_val)

In [218]:
df_scaled.to_csv('proc_data.csv')   #Simply saving the data

# FEATURE SELECTION AND ENGINEERING
As it has been presented in the Feature Exploration section, many (pairs of) features tend to have strong correlation to the target and some features were target leakages.\
In this section, features are filtered and refined features are engineered using Dimensionality Reduction method.

Removing the most obvious target leakages:

In [219]:
X = df_scaled.drop(["ID", "Status", "N_Days", "Prothrombin", "Albumin", "Spiders", "Ascites", "Edema", "Platelets"], axis=1)
y = X.pop("Stage")
y = y - 1
y.astype(int)

In [220]:
X_val = df_scaled_val.drop(["ID", "Status", "N_Days", "Prothrombin", "Albumin", "Spiders", "Ascites", "Edema", "Platelets"], axis=1)
y_val = X_val.pop("Stage")
y_val = y_val - 1
y_val.astype(int)

Decrementing the target class by 1.\
XGBClassifier expects target classes to be in the range of `(0, number of class]`.

In [221]:
from sklearn.decomposition import IncrementalPCA

def Dimen_Reduce(X, y, n, X_val):
    
    pca = IncrementalPCA(n_components = n, batch_size=100)
    X_pca = pca.fit_transform(X, y)
    V_pca = pca.transform(X_val)
    
    return X_pca, V_pca


Performing Dimensionality Reduction using Incremental Principal Component Analysis\
Depending on the size of the input data, this algorithm can be much more memory efficient than a PCA.\
\
This algorithm has constant memory complexity, on the order of `batch_size * n_features`, enabling use of `np.memmap files` without loading the entire file into memory.\
(For more information, please check [documentation](http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.IncrementalPCA.html))

Based on insights gained from Feature Exploration, new features are engineered using few selected features which might show stronger correlation to the target.

In [222]:
new_comp1 = X[["Bilirubin", "Alk_Phos"]]
new_comp2 = X[["Bilirubin", "SGOT"]]
new_comp3 = X[["Drug", "NaN_Drug", "Copper"]]
new_comp4 = X[["Tryglicerides", "Bilirubin", "Alk_Phos"]]

new_comp1_val = X_val[["Bilirubin", "Alk_Phos"]]
new_comp2_val = X_val[["Bilirubin", "SGOT"]]
new_comp3_val = X_val[["Drug", "NaN_Drug", "Copper"]]
new_comp4_val = X_val[["Tryglicerides", "Bilirubin", "Alk_Phos"]]

new_comps1, new_comps1_val = Dimen_Reduce(new_comp1, y, 2, new_comp1_val)
new_comps2, new_comps2_val = Dimen_Reduce(new_comp2, y, 2, new_comp2_val)
new_comps3, new_comps3_val = Dimen_Reduce(new_comp3, y, 2, new_comp3_val)
new_comps4, new_comps4_val = Dimen_Reduce(new_comp4, y, 3, new_comp4_val)


X[["new_comp1", "new_comp2"]] = new_comps1
X[["new_comp3", "new_comp4"]] = new_comps2
X[["new_comp5", "new_comp6"]] = new_comps3
X[["new_comp7", "new_comp8", "new_comp9"]] = new_comps4

X_val[["new_comp1", "new_comp2"]] = new_comps1_val
X_val[["new_comp3", "new_comp4"]] = new_comps2_val
X_val[["new_comp5", "new_comp6"]] = new_comps3_val
X_val[["new_comp7", "new_comp8", "new_comp9"]] = new_comps4_val

X_val

Finding Mutual Information scores between features to see how strongly they are correlated to the target.

In [223]:
discrete_features = X.dtypes is not str

from sklearn.feature_selection import mutual_info_classif


def make_mi_scores(X, y, discrete_features):
    mi_scores = mutual_info_classif(X, y, discrete_features=discrete_features, random_state = 42)
    mi_scores = pd.Series(mi_scores, name="MI Scores", index=X.columns)
    mi_scores = mi_scores.sort_values(ascending=False)
    return mi_scores

mi_scores = make_mi_scores(X, y, discrete_features)
mi_scores = mi_scores.sort_values(ascending=False)
k = mi_scores.index.tolist()
refined_cols = k[:13]
print(refined_cols)
mi_scores[:15]

In [224]:
import matplotlib.pyplot as plt
import numpy as np

def plot_mi_scores(scores):
    scores = scores.sort_values(ascending=True)
    width = np.arange(len(scores))
    ticks = list(scores.index)
    plt.barh(width, scores)
    plt.yticks(width, ticks)
    plt.title("Mutual Information Scores")

plt.figure(dpi=100, figsize=(8, 5))
plot_mi_scores(mi_scores)

Using 12 columns with top mutual information scores for training.

# MODELING

In [225]:
"""%%time
from numpy import mean
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RepeatedStratifiedKFold
from xgboost import XGBClassifier

# define model
model = XGBClassifier(objective = "multi:softmax" , num_class = 4, n_jobs = -1, 
                    tree_method = 'gpu_hist', random_state = 42, use_label_encoder=False)
# define grid
weights = [400]
param_grid = {'learning_rate': [0.185, 0.5, 0.75, 0.95], 'max_depth': [], 'n_estimators': []}
# define evaluation procedure
cv = RepeatedStratifiedKFold(n_splits=4 , n_repeats= 1, random_state=42)
# define grid search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=cv, scoring='f1_weighted')
# execute the grid search
grid_result = grid.fit(X, y)
# report the best configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# report all configurations
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
   print("%f (%f) with: %r" % (mean, stdev, param))"""

In [226]:
"""from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

# confusion matrix
matrix = confusion_matrix(y_test, results1, labels=[0, 1, 2, 3])
print('Confusion matrix : \n',matrix)

# outcome values order in sklearn
tp, fn, fp, tn = confusion_matrix(y_test, results1,labels=[1,0]).reshape(-1)
print('Outcome values : \n', tp, fn, fp, tn)

# classification report for precision, recall f1-score and accuracy
matrix = classification_report(y_test, results1,labels=[0, 1, 2, 3])
print('Classification report : \n',matrix)"""

Splitting data for training and testing and checking overfitting and underfitting.\
Using `stratify` parameter to account for the imbalance in the data.

Tests on `XGBClassifier` with `OneVsRestClassifier`:

In [230]:
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import f1_score


sizes = [0.2, 0.23, 0.25, 0.28, 0.3, 0.33, 0.35, 0.38, 0.4]
score_train = []
score_test = []
score_val = []

"""obj = XGBClassifier(objective = "multi:softmax" , num_class = 4, n_jobs = -1, learning_rate= 0.17,
                    max_depth = 4, n_estimators = 150, tree_method = 'gpu_hist', subsample = 0.95,
                    random_state = 42, use_label_encoder=False, eval_metric = "logloss")"""

from sklearn.multiclass import OneVsRestClassifier

obj = OneVsRestClassifier(XGBClassifier(n_jobs = -1, learning_rate= 0.17,
                    max_depth = 4, n_estimators = 150, tree_method = 'gpu_hist', subsample = 0.95, scale_pos_weight = 0.15,
                    random_state = 42, use_label_encoder=False, eval_metric = "logloss"))

for size in sizes:
    xgb1 = obj    
    X_train, X_test, y_train, y_test = train_test_split(X[refined_cols], y, test_size=size, random_state=42, stratify=y)
    xgb1.fit(X_train, y_train)
    train_results = xgb1.predict(X_train)
    test_results = xgb1.predict(X_test)
    val_results = xgb1.predict(X_val[refined_cols])
    acc_train = f1_score(y_train,train_results, average = 'weighted')
    acc_test = f1_score(y_test,test_results, average = 'weighted')
    acc_val = f1_score(y_val,val_results, average = 'weighted')
    score_train.append(acc_train)
    score_test.append(acc_test)
    score_val.append(acc_val)
    del xgb1
    
import matplotlib.pyplot as plt
import numpy as np

ypoints_train = np.array(score_train)
ypoints_test = np.array(score_test)
ypoints_val = np.array(score_val)

x = sizes
plt.figure(figsize=(10, 6))
plt.plot(x, ypoints_train, color = 'r', label = "training score")
plt.plot(x, ypoints_test, color = 'g', label = "testing score")
plt.plot(x, ypoints_val, color = 'b', label = "validation score")
plt.xlim([0.2, 0.4])
plt.ylim([0, 1])
plt.xlabel("Sizes")
plt.ylabel("F1 Scores")
plt.title("XGBoost Classifier Performance")
plt.legend()
plt.show()

for i in range(0, len(sizes)):
    print("For size : {} - Training score : {} and Testing score : {} and Validation score : {}".format(sizes[i], score_train[i].round(4), score_test[i].round(4), score_val[i].round(4)))
    
print("\nAverage Training: {} and Average Testing: {} and Average Validation: {}".format(ypoints_train.mean().round(4), ypoints_test.mean().round(4), ypoints_val.mean().round(4) ))
i = ypoints_train.argmax()
print("\nMaximum Training: {} and respective Testing {} at size {} Validation {} at size {} ".format(np.amax(ypoints_train).round(4), score_test[i].round(4), score_val[i].round(4), sizes[i]))
j = ypoints_test.argmax()
print("Maximum Testing: {} and respective Training {} and Validation {} at size {} ".format(np.amax(ypoints_test).round(4), score_train[j].round(4), score_val[j].round(4), sizes[j]))
k = ypoints_val.argmax()
print("Maximum Validation: {} and respective Training {} and Testing {} at size {} ".format(np.amax(ypoints_val).round(4), score_train[k].round(4), score_test[k].round(4), sizes[k]))

Tests on `RandomForestClassifier` with `OneVsRestClassifier`:

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score


sizes = [0.2, 0.23, 0.25, 0.28, 0.3, 0.33, 0.35, 0.38, 0.4]
score_train = []
score_test = []

"""obj = RandomForestClassifier(n_jobs = -1, n_estimators = 100, max_depth = 3, random_state=42)"""

from sklearn.multiclass import OneVsRestClassifier

obj = OneVsRestClassifier(RandomForestClassifier(n_jobs = -1, n_estimators = 200, max_depth = 6, random_state=42))


for size in sizes:
    xgb1 = obj    
    X_train, X_test, y_train, y_test = train_test_split(X[refined_cols], y, test_size=size, random_state=42, stratify=y)
    xgb1.fit(X_train, y_train)
    train_results = xgb1.predict(X_train)
    test_results = xgb1.predict(X_test)
    acc_train = f1_score(y_train,train_results, average = 'weighted')
    acc_test = f1_score(y_test,test_results, average = 'weighted')
    score_train.append(acc_train)
    score_test.append(acc_test)
    del xgb1
    
import matplotlib.pyplot as plt
import numpy as np

ypoints_train = np.array(score_train)
ypoints_test = np.array(score_test)

x = sizes
plt.figure(figsize=(10, 6))
plt.plot(x, ypoints_train, color = 'r', label = "training score")
plt.plot(x, ypoints_test, color = 'g', label = "testing score")
plt.xlim([0.2, 0.4])
plt.ylim([0, 1])
plt.xlabel("Sizes")
plt.ylabel("F1 Scores")
plt.title("Random Forest Classifier Performance")
plt.legend()
plt.show()

for i in range(0, len(sizes)):
    print("For size : {} - Training score : {} and Testing score : {}".format(sizes[i], score_train[i].round(4), score_test[i].round(4)))
    
print("\nAverage Training: {} and Average Testing: {} ".format(ypoints_train.mean().round(4), ypoints_test.mean().round(4)))
i = ypoints_train.argmax()
print("\nMaximum Training: {} and respective Testing {} at size {} ".format(np.amax(ypoints_train).round(4), score_test[i].round(4), sizes[i]))
j = ypoints_train.argmin()
print("Maximum Testing: {} and respective Training {} at size {} ".format(np.amax(ypoints_test).round(4), score_train[j].round(4), sizes[j]))

Usually in medical research, atmost 5-6% of missing values in the dataset is acceptable. \
Since the given dataset has 20.63% of the values missing, the testing scores are consequentially low.

**Acknowledgements:**
* https://medlineplus.gov/lab-tests/alkaline-phosphatase/
* https://www.webmd.com/heart-disease/heart-failure/edema-overview#:~:text=Medications%2C%20pregnancy%2C%20infections%2C%20and,almost%20anywhere%20in%20the%20body.
* https://www.webmd.com/digestive-disorders/tests-for-cirrhosis
* https://www.hepatitis.va.gov/hcv/patient/diagnosis/labtests-prothrombin-time.asp#:~:text=When%20the%20PT%20is%20high,serious%20liver%20damage%20or%20cirrhosis.
* https://amj.amegroups.com/article/view/4703/html
* https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3644745/#:~:text=found%20that%20the%20serum%20cholesterol,the%20progress%20of%20alcoholic%20cirrhosis.&text=Few%20studies%20on%20cirrhosis%20of,cholesterol%20values%20were%20significantly%20diminished.
* https://www.webmd.com/hepatitis/enlarged-liver-causes
* https://www.healthgrades.com/right-care/liver-conditions/enlarged-liver
* https://www.journal-of-hepatology.eu/article/S0168-8278(11)00682-9/fulltext
* https://gi.org/topics/ascites/
* https://www.healthline.com/health/sgot-test#followup
* https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3644745/#:~:text=Severe%20metabolic%20impairment%20in%20cirrhosis,density%20lipoprotein%20(LDL)%20cholesterol.
